In [1]:
import pandas as pd
import numpy as np

In [2]:
dfs = pd.read_csv("Chest_xray_Corona_dataset_Summary.csv")

In [3]:
dfs.head()

,Unnamed: 0,Label,Label_1_Virus_category,Label_2_Virus_category,Image_Count
0,0,Normal,NaN,NaN,1576
1,1,Pnemonia,Stress-Smoking,ARDS,2
2,2,Pnemonia,Virus,NaN,1493
3,3,Pnemonia,Virus,COVID-19,58
4,4,Pnemonia,Virus,SARS,4


In [4]:
# loading metadata
df = pd.read_csv("Chest_xray_Corona_Metadata.csv")
df.head()

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN


In [5]:
# we are not going to focus on the virus category because we need to classify the images as nornam vs pneumonia
df = df[["X_ray_image_name", "Label", "Dataset_type"]]
df = df.dropna()

In [6]:
df.head()

,X_ray_image_name,Label,Dataset_type
0,IM-0128-0001.jpeg,Normal,TRAIN
1,IM-0127-0001.jpeg,Normal,TRAIN
2,IM-0125-0001.jpeg,Normal,TRAIN
3,IM-0122-0001.jpeg,Normal,TRAIN
4,IM-0119-0001.jpeg,Normal,TRAIN


In [7]:
df.tail()

,X_ray_image_name,Label,Dataset_type
5905,person1637_virus_2834.jpeg,Pnemonia,TEST
5906,person1635_virus_2831.jpeg,Pnemonia,TEST
5907,person1634_virus_2830.jpeg,Pnemonia,TEST
5908,person1633_virus_2829.jpeg,Pnemonia,TEST
5909,person1632_virus_2827.jpeg,Pnemonia,TEST


In [8]:
# tqdm is a libraby for progress bar in python... it's not compulsory to use this... I am just giving it a try
from tqdm import tqdm

# we will be introducing new column to our frame... This will store the image location for the respective images
df["File_Path"] = 0

# the base path to images
base_path = "D:/adity/Projects/Chest_X_Ray_Classifier/Coronahack-Chest-XRay-Dataset/"

for index in tqdm(df.index):
    # now we will decide if the image is in test dir or train dir
    if df.loc[index, "Dataset_type"] == "TRAIN":
        path = base_path + "train/"
    else:
        path = base_path + "test/"
    # now we shall write the path to img
    df.loc[index, "File_Path"] = path + df.loc[index, "X_ray_image_name"]

100%|████████████████████████████████████████████████████████████████████████████| 5910/5910 [00:05<00:00, 1154.89it/s]


In [9]:
df.head()

,X_ray_image_name,Label,Dataset_type,File_Path
0,IM-0128-0001.jpeg,Normal,TRAIN,D:/adity/Projects/Chest_X_Ray_Classifier/Coron...
1,IM-0127-0001.jpeg,Normal,TRAIN,D:/adity/Projects/Chest_X_Ray_Classifier/Coron...
2,IM-0125-0001.jpeg,Normal,TRAIN,D:/adity/Projects/Chest_X_Ray_Classifier/Coron...
3,IM-0122-0001.jpeg,Normal,TRAIN,D:/adity/Projects/Chest_X_Ray_Classifier/Coron...
4,IM-0119-0001.jpeg,Normal,TRAIN,D:/adity/Projects/Chest_X_Ray_Classifier/Coron...


In [30]:
# Let's begin preprocessing

# importing my CNN
from my_model.Doctor import Doctor

# importing required packages
import matplotlib
matplotlib.use(backend="Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import np_utils

import matplotlib.pyplot as plt
import pickle
import cv2
import os

In [13]:
# so they are using around 10% images for testing
# lets do what they want

trainX = []
trainY = []
testX = []
testY = []

for index in tqdm(df.index):
    
    image = cv2.imread(df.loc[index, "File_Path"])
    image = cv2.resize(image, (64, 64))
    
    if df.loc[index, "Dataset_type"] == "TRAIN":
        trainX.append(image)
        trainY.append(df.loc[index, "Label"])
    else:
        testX.append(image)
        testY.append(df.loc[index, "Label"])

print(len(trainX), len(trainY))
print(len(testX), len(testY))

100%|██████████████████████████████████████████████████████████████████████████████| 5910/5910 [01:30<00:00, 64.98it/s]


5286 5286
624 624


In [14]:
# converting my dataset into numpy array and preprocess by scaling pixel intensities to range [0, 1]

# train dataset
trainX = np.array(trainX, dtype="float") / 255.0

# test dataset
testX = np.array(testX, dtype="float") / 255.0

print(len(trainX), len(testX))

5286 624


In [15]:
# Encoding the train labels currently as strings, to integers and then one-hot encode them

train_le = LabelEncoder()
trainY = train_le.fit_transform(trainY)
trainY = np_utils.to_categorical(y=trainY, num_classes=len(set(trainY)))

# Encoding the test labels currently as strings, to integers and then one-hot encode them

test_le = LabelEncoder()
testY = test_le.fit_transform(testY)
testY = np_utils.to_categorical(y=testY, num_classes=len(set(testY)))

print(len(trainY), len(testY))

5286 624


In [16]:
# construct the training image generator for data augmentation
# aug will be used further to generate images from our data

aug = ImageDataGenerator(rotation_range=36, zoom_range=0.2, width_shift_range=0.25, height_shift_range=0.25, shear_range=0.2, horizontal_flip=True, fill_mode="nearest")

In [17]:
learn_rate = 1e-4
batch_size = 15
epochs = 75

In [18]:
# Initializing the optimizer and compiling model

print("[INFO] Compiling Model...")
opt = Adam(learning_rate=learn_rate, decay=learn_rate/epochs)
model = Doctor.classify(width=64, height=64, depth=3, classes=len(train_le.classes_))
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] Compiling Model...


In [19]:
# Let's finally train our network

print("[INFO] Training network for: {} epochs...".format(epochs))
res = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size), validation_data=(testX, testY), steps_per_epoch=len(trainX) // batch_size, epochs=epochs)

[INFO] Training network for: 75 epochs...
Epoch 1/75
352/352 [==============================] - ETA: 24:13 - loss: 7.2983 - accuracy: 0.300 - ETA: 8:08 - loss: 6.2759 - accuracy: 0.377 - ETA: 4:06 - loss: 6.5907 - accuracy: 0.38 - ETA: 2:45 - loss: 6.2805 - accuracy: 0.39 - ETA: 2:08 - loss: 5.9706 - accuracy: 0.38 - ETA: 1:50 - loss: 6.0089 - accuracy: 0.40 - ETA: 1:31 - loss: 6.0238 - accuracy: 0.39 - ETA: 1:18 - loss: 5.9937 - accuracy: 0.39 - ETA: 1:08 - loss: 6.0912 - accuracy: 0.39 - ETA: 1:03 - loss: 6.1497 - accuracy: 0.39 - ETA: 59s - loss: 6.0879 - accuracy: 0.3988 - ETA: 53s - loss: 6.0909 - accuracy: 0.395 - ETA: 50s - loss: 6.0312 - accuracy: 0.399 - ETA: 46s - loss: 5.9348 - accuracy: 0.404 - ETA: 42s - loss: 5.8496 - accuracy: 0.407 - ETA: 40s - loss: 5.8841 - accuracy: 0.404 - ETA: 38s - loss: 5.8349 - accuracy: 0.407 - ETA: 37s - loss: 5.8243 - accuracy: 0.411 - ETA: 35s - loss: 5.8323 - accuracy: 0.409 - ETA: 33s - loss: 5.8610 - accuracy: 0.409 - ETA: 31s - loss: 5.8

352/352 [==============================] - ETA: 8s - loss: 6.7070 - accuracy: 0.43 - ETA: 18s - loss: 5.2578 - accuracy: 0.466 - ETA: 13s - loss: 5.5338 - accuracy: 0.441 - ETA: 12s - loss: 5.1329 - accuracy: 0.450 - ETA: 11s - loss: 5.2313 - accuracy: 0.450 - ETA: 10s - loss: 4.9316 - accuracy: 0.456 - ETA: 10s - loss: 4.9629 - accuracy: 0.444 - ETA: 10s - loss: 5.0017 - accuracy: 0.447 - ETA: 9s - loss: 4.9210 - accuracy: 0.447 - ETA: 9s - loss: 4.7369 - accuracy: 0.45 - ETA: 9s - loss: 4.8681 - accuracy: 0.45 - ETA: 9s - loss: 4.8285 - accuracy: 0.45 - ETA: 9s - loss: 4.8413 - accuracy: 0.44 - ETA: 9s - loss: 4.7231 - accuracy: 0.45 - ETA: 8s - loss: 4.7157 - accuracy: 0.45 - ETA: 8s - loss: 4.7385 - accuracy: 0.44 - ETA: 8s - loss: 4.6986 - accuracy: 0.45 - ETA: 8s - loss: 4.7168 - accuracy: 0.44 - ETA: 8s - loss: 4.7499 - accuracy: 0.44 - ETA: 8s - loss: 4.7458 - accuracy: 0.44 - ETA: 8s - loss: 4.7001 - accuracy: 0.44 - ETA: 8s - loss: 4.6586 - accuracy: 0.44 - ETA: 8s - loss: 4.

352/352 [==============================] - ETA: 8s - loss: 5.3686 - accuracy: 0.40 - ETA: 10s - loss: 4.8004 - accuracy: 0.400 - ETA: 9s - loss: 4.5109 - accuracy: 0.406 - ETA: 9s - loss: 4.6667 - accuracy: 0.42 - ETA: 9s - loss: 4.9502 - accuracy: 0.39 - ETA: 8s - loss: 4.8443 - accuracy: 0.41 - ETA: 8s - loss: 4.7452 - accuracy: 0.41 - ETA: 8s - loss: 4.9987 - accuracy: 0.41 - ETA: 8s - loss: 4.7431 - accuracy: 0.42 - ETA: 8s - loss: 4.7216 - accuracy: 0.42 - ETA: 8s - loss: 4.6998 - accuracy: 0.42 - ETA: 8s - loss: 4.8631 - accuracy: 0.42 - ETA: 8s - loss: 4.7838 - accuracy: 0.42 - ETA: 8s - loss: 4.7882 - accuracy: 0.42 - ETA: 8s - loss: 4.8062 - accuracy: 0.42 - ETA: 8s - loss: 4.8439 - accuracy: 0.42 - ETA: 8s - loss: 4.8804 - accuracy: 0.42 - ETA: 8s - loss: 4.8931 - accuracy: 0.41 - ETA: 7s - loss: 4.9204 - accuracy: 0.41 - ETA: 7s - loss: 4.7940 - accuracy: 0.42 - ETA: 7s - loss: 4.8436 - accuracy: 0.42 - ETA: 7s - loss: 4.8252 - accuracy: 0.42 - ETA: 7s - loss: 4.7538 - accur

352/352 [==============================] - ETA: 8s - loss: 5.2811 - accuracy: 0.36 - ETA: 9s - loss: 4.4882 - accuracy: 0.44 - ETA: 9s - loss: 3.8203 - accuracy: 0.46 - ETA: 9s - loss: 3.8878 - accuracy: 0.43 - ETA: 9s - loss: 4.3116 - accuracy: 0.43 - ETA: 9s - loss: 4.4483 - accuracy: 0.43 - ETA: 9s - loss: 4.5426 - accuracy: 0.42 - ETA: 8s - loss: 4.3608 - accuracy: 0.43 - ETA: 8s - loss: 4.3690 - accuracy: 0.44 - ETA: 8s - loss: 4.4843 - accuracy: 0.44 - ETA: 8s - loss: 4.5817 - accuracy: 0.44 - ETA: 8s - loss: 4.5076 - accuracy: 0.44 - ETA: 8s - loss: 4.5432 - accuracy: 0.44 - ETA: 8s - loss: 4.5363 - accuracy: 0.44 - ETA: 8s - loss: 4.5743 - accuracy: 0.44 - ETA: 8s - loss: 4.4449 - accuracy: 0.44 - ETA: 8s - loss: 4.3060 - accuracy: 0.45 - ETA: 8s - loss: 4.3825 - accuracy: 0.44 - ETA: 8s - loss: 4.4521 - accuracy: 0.44 - ETA: 8s - loss: 4.4670 - accuracy: 0.44 - ETA: 8s - loss: 4.4798 - accuracy: 0.44 - ETA: 7s - loss: 4.3561 - accuracy: 0.45 - ETA: 7s - loss: 4.2916 - accuracy

352/352 [==============================] - ETA: 8s - loss: 4.3131 - accuracy: 0.53 - ETA: 8s - loss: 3.6414 - accuracy: 0.51 - ETA: 10s - loss: 3.9960 - accuracy: 0.480 - ETA: 10s - loss: 4.0150 - accuracy: 0.457 - ETA: 9s - loss: 4.0691 - accuracy: 0.451 - ETA: 9s - loss: 3.9011 - accuracy: 0.45 - ETA: 9s - loss: 3.9837 - accuracy: 0.45 - ETA: 9s - loss: 4.0417 - accuracy: 0.45 - ETA: 9s - loss: 4.0593 - accuracy: 0.45 - ETA: 8s - loss: 4.1836 - accuracy: 0.44 - ETA: 8s - loss: 4.2104 - accuracy: 0.44 - ETA: 8s - loss: 4.3783 - accuracy: 0.44 - ETA: 8s - loss: 4.3315 - accuracy: 0.43 - ETA: 8s - loss: 4.4350 - accuracy: 0.43 - ETA: 8s - loss: 4.5014 - accuracy: 0.43 - ETA: 8s - loss: 4.4917 - accuracy: 0.43 - ETA: 8s - loss: 4.5254 - accuracy: 0.43 - ETA: 8s - loss: 4.5636 - accuracy: 0.43 - ETA: 8s - loss: 4.5349 - accuracy: 0.43 - ETA: 7s - loss: 4.5637 - accuracy: 0.43 - ETA: 7s - loss: 4.5232 - accuracy: 0.43 - ETA: 7s - loss: 4.5126 - accuracy: 0.43 - ETA: 7s - loss: 4.5414 - acc

...

352/352 [==============================] - ETA: 8s - loss: 5.3638 - accuracy: 0.36 - ETA: 8s - loss: 3.4954 - accuracy: 0.46 - ETA: 8s - loss: 3.2491 - accuracy: 0.47 - ETA: 8s - loss: 2.9052 - accuracy: 0.50 - ETA: 8s - loss: 3.0681 - accuracy: 0.49 - ETA: 8s - loss: 3.0026 - accuracy: 0.49 - ETA: 8s - loss: 2.9836 - accuracy: 0.49 - ETA: 8s - loss: 2.8142 - accuracy: 0.49 - ETA: 8s - loss: 2.7715 - accuracy: 0.50 - ETA: 8s - loss: 2.7217 - accuracy: 0.49 - ETA: 8s - loss: 2.6584 - accuracy: 0.49 - ETA: 8s - loss: 2.7132 - accuracy: 0.49 - ETA: 8s - loss: 2.7345 - accuracy: 0.49 - ETA: 8s - loss: 2.6896 - accuracy: 0.49 - ETA: 8s - loss: 2.7156 - accuracy: 0.49 - ETA: 7s - loss: 2.7533 - accuracy: 0.49 - ETA: 7s - loss: 2.7737 - accuracy: 0.50 - ETA: 7s - loss: 2.7818 - accuracy: 0.50 - ETA: 7s - loss: 2.8070 - accuracy: 0.49 - ETA: 7s - loss: 2.7852 - accuracy: 0.49 - ETA: 7s - loss: 2.7669 - accuracy: 0.49 - ETA: 7s - loss: 2.7428 - accuracy: 0.49 - ETA: 7s - loss: 2.7001 - accuracy

352/352 [==============================] - ETA: 8s - loss: 4.1256 - accuracy: 0.46 - ETA: 8s - loss: 2.5104 - accuracy: 0.55 - ETA: 8s - loss: 2.2657 - accuracy: 0.56 - ETA: 8s - loss: 2.6956 - accuracy: 0.52 - ETA: 8s - loss: 3.0589 - accuracy: 0.49 - ETA: 8s - loss: 2.8349 - accuracy: 0.50 - ETA: 8s - loss: 2.7698 - accuracy: 0.50 - ETA: 8s - loss: 2.7554 - accuracy: 0.51 - ETA: 8s - loss: 2.7144 - accuracy: 0.51 - ETA: 8s - loss: 2.8589 - accuracy: 0.50 - ETA: 8s - loss: 2.7308 - accuracy: 0.51 - ETA: 8s - loss: 2.7247 - accuracy: 0.51 - ETA: 8s - loss: 2.8032 - accuracy: 0.50 - ETA: 8s - loss: 2.8256 - accuracy: 0.50 - ETA: 7s - loss: 2.8322 - accuracy: 0.50 - ETA: 7s - loss: 2.8261 - accuracy: 0.50 - ETA: 7s - loss: 2.8085 - accuracy: 0.50 - ETA: 7s - loss: 2.8259 - accuracy: 0.50 - ETA: 7s - loss: 2.8804 - accuracy: 0.50 - ETA: 7s - loss: 2.8333 - accuracy: 0.50 - ETA: 7s - loss: 2.7681 - accuracy: 0.50 - ETA: 7s - loss: 2.7423 - accuracy: 0.50 - ETA: 7s - loss: 2.7399 - accuracy

352/352 [==============================] - ETA: 8s - loss: 2.0701 - accuracy: 0.56 - ETA: 8s - loss: 2.5084 - accuracy: 0.55 - ETA: 8s - loss: 2.6414 - accuracy: 0.51 - ETA: 8s - loss: 2.7787 - accuracy: 0.50 - ETA: 8s - loss: 2.9598 - accuracy: 0.49 - ETA: 8s - loss: 3.2018 - accuracy: 0.48 - ETA: 8s - loss: 3.0557 - accuracy: 0.48 - ETA: 8s - loss: 2.8311 - accuracy: 0.49 - ETA: 8s - loss: 2.7560 - accuracy: 0.49 - ETA: 8s - loss: 2.6078 - accuracy: 0.49 - ETA: 8s - loss: 2.5933 - accuracy: 0.49 - ETA: 8s - loss: 2.6524 - accuracy: 0.49 - ETA: 8s - loss: 2.6107 - accuracy: 0.50 - ETA: 8s - loss: 2.5537 - accuracy: 0.50 - ETA: 8s - loss: 2.5518 - accuracy: 0.50 - ETA: 8s - loss: 2.5557 - accuracy: 0.50 - ETA: 7s - loss: 2.6765 - accuracy: 0.49 - ETA: 7s - loss: 2.6495 - accuracy: 0.49 - ETA: 7s - loss: 2.6649 - accuracy: 0.49 - ETA: 7s - loss: 2.6561 - accuracy: 0.49 - ETA: 7s - loss: 2.7042 - accuracy: 0.49 - ETA: 7s - loss: 2.7261 - accuracy: 0.49 - ETA: 7s - loss: 2.6913 - accuracy

352/352 [==============================] - ETA: 9s - loss: 3.7787 - accuracy: 0.46 - ETA: 9s - loss: 3.0127 - accuracy: 0.50 - ETA: 8s - loss: 3.0173 - accuracy: 0.50 - ETA: 8s - loss: 3.7051 - accuracy: 0.47 - ETA: 8s - loss: 3.5530 - accuracy: 0.46 - ETA: 8s - loss: 2.9733 - accuracy: 0.48 - ETA: 8s - loss: 3.0941 - accuracy: 0.47 - ETA: 8s - loss: 3.1088 - accuracy: 0.47 - ETA: 8s - loss: 3.1409 - accuracy: 0.47 - ETA: 8s - loss: 3.1453 - accuracy: 0.47 - ETA: 8s - loss: 3.0792 - accuracy: 0.47 - ETA: 8s - loss: 3.1055 - accuracy: 0.47 - ETA: 8s - loss: 3.1152 - accuracy: 0.47 - ETA: 8s - loss: 3.0465 - accuracy: 0.47 - ETA: 8s - loss: 3.0759 - accuracy: 0.47 - ETA: 8s - loss: 3.1865 - accuracy: 0.46 - ETA: 8s - loss: 3.1528 - accuracy: 0.47 - ETA: 7s - loss: 3.0751 - accuracy: 0.47 - ETA: 7s - loss: 3.0750 - accuracy: 0.47 - ETA: 7s - loss: 3.0903 - accuracy: 0.47 - ETA: 7s - loss: 3.1361 - accuracy: 0.47 - ETA: 7s - loss: 3.1796 - accuracy: 0.47 - ETA: 7s - loss: 3.1466 - accuracy

352/352 [==============================] - ETA: 8s - loss: 2.1357 - accuracy: 0.50 - ETA: 8s - loss: 2.3042 - accuracy: 0.54 - ETA: 8s - loss: 2.4168 - accuracy: 0.52 - ETA: 8s - loss: 2.5535 - accuracy: 0.51 - ETA: 8s - loss: 2.7269 - accuracy: 0.51 - ETA: 8s - loss: 2.7508 - accuracy: 0.51 - ETA: 8s - loss: 2.7067 - accuracy: 0.51 - ETA: 8s - loss: 2.5494 - accuracy: 0.51 - ETA: 8s - loss: 2.4501 - accuracy: 0.52 - ETA: 8s - loss: 2.6235 - accuracy: 0.51 - ETA: 8s - loss: 2.7779 - accuracy: 0.51 - ETA: 8s - loss: 2.8261 - accuracy: 0.51 - ETA: 8s - loss: 2.8330 - accuracy: 0.51 - ETA: 8s - loss: 2.8057 - accuracy: 0.51 - ETA: 8s - loss: 2.8177 - accuracy: 0.51 - ETA: 7s - loss: 2.7496 - accuracy: 0.51 - ETA: 7s - loss: 2.8286 - accuracy: 0.51 - ETA: 7s - loss: 2.8521 - accuracy: 0.51 - ETA: 7s - loss: 2.8592 - accuracy: 0.50 - ETA: 7s - loss: 2.8278 - accuracy: 0.50 - ETA: 7s - loss: 2.8192 - accuracy: 0.50 - ETA: 7s - loss: 2.8180 - accuracy: 0.50 - ETA: 7s - loss: 2.8335 - accuracy

In [20]:
# Evaluating the Network

print("[INFO] Evaluating the Network...")
preds = model.predict(testX, batch_size=batch_size)
print(classification_report(testY.argmax(axis=1), preds.argmax(axis=1), target_names=test_le.classes_))

[INFO] Evaluating the Network...
              precision    recall  f1-score   support

      Normal       0.71      0.79      0.74       234
    Pnemonia       0.86      0.81      0.83       390

    accuracy                           0.80       624
   macro avg       0.79      0.80      0.79       624
weighted avg       0.80      0.80      0.80       624



In [22]:
# we have a score of around 80%
# Save the Network i.e Model to Disk as it is more efficient

print("Serializing Network to disk")
model.save("doctor.model")

Serializing Network to disk


In [23]:
# Saving the Label Encoder to disk as well

print("Serializing Label Encoder to disk")
f = open("le.pickle", "wb")
f.write(pickle.dumps(train_le))
f.close()

Serializing Label Encoder to disk


In [31]:
#lets plot the training loss and accuracy for re-trained model

print("[INFO] Serializing the plotted graph of trained model to disk...")
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, epochs), res.history["loss"], label="train_loss")
plt.plot(np.arange(0, epochs), res.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, epochs), res.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, epochs), res.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")
print("[INFO] Saved plot as plot.png...")

[INFO] Serializing the plotted graph of trained model to disk...
[INFO] Saved plot as plot.png...
